<a href="https://colab.research.google.com/github/ferdinand-lanvino/AES-Encryption-Classes/blob/master/commodity_predictor_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup** <a name="setup"></a>

Clone the dataset

In [1]:
# Clone commodity-predictor repo.
!git clone -l -s git://github.com/ferdinand-lanvino/commodity-predictor.git
%cd commodity-predictor
!ls

Cloning into 'commodity-predictor'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 136 (delta 52), reused 58 (delta 15), pack-reused 0
Receiving objects: 100% (136/136), 1.74 MiB | 1.91 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/commodity-predictor
commodity_predictor_research.ipynb  Dataset
commodity-price-predictor.ipynb     README.md


Pull the updates if needed

In [3]:
!git pull

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 1), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (5/5), done.
From git://github.com/ferdinand-lanvino/commodity-predictor
   9708b4b..b2fbcf4  master     -> origin/master
Updating 9708b4b..b2fbcf4
Fast-forward
 Dataset/Converted/commodity.csv | 29333 ++++++++++++++++++++++++++++++++++++++
 1 file changed, 29333 insertions(+)
 create mode 100644 Dataset/Converted/commodity.csv


Import Dependency

In [2]:
#basic packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import datetime # manipulating date formats

#visualization
import matplotlib.pylab as plt # basic plotting
import seaborn as sns # for prettier plots

#statistic
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## **Exploratory Data Analysis** <a name="exploratory-data-analysis"></a>

Import combined dataset located in Dataset/Converted/commodity.csv

In [11]:
#Importing dataset 
dataset = pd.read_csv('Dataset/Converted/commodity.csv')
dataset['tanggal'] = pd.to_datetime(dataset['tanggal'],format='%d/%m/%Y')
print(dataset.info())
print(dataset)
#Get bawang merah dataset
bawang_merah = dataset[dataset.komoditas=='Bawang Merah']
bawang_merah = bawang_merah[['tanggal','harga']].set_index(['tanggal'])
#bawang_merah = bawang_merah.fillna(0)
bawang_merah.index = pd.DatetimeIndex(bawang_merah.index)
print(bawang_merah.info())
#imputation
idx = pd.date_range('25/07/2016', '29-05-2020')
bawang_merah.index = pd.DatetimeIndex(bawang_merah.index)
reindexedBM = bawang_merah.reindex(idx, fill_value=np.nan)
print(bawang_merah)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29332 entries, 0 to 29331
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   komoditas  29332 non-null  object        
 1   tanggal    29332 non-null  datetime64[ns]
 2   harga      29242 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 687.6+ KB
None
              komoditas    tanggal    harga
0                 Beras 2016-07-25  10900.0
1                 Beras 2016-07-26  10950.0
2                 Beras 2016-07-27  10950.0
3                 Beras 2016-07-28  10950.0
4                 Beras 2016-07-29  10950.0
...                 ...        ...      ...
29327  Gula Pasir Lokal 2020-05-20  17500.0
29328  Gula Pasir Lokal 2020-05-26  17650.0
29329  Gula Pasir Lokal 2020-05-27  17650.0
29330  Gula Pasir Lokal 2020-05-28  17650.0
29331  Gula Pasir Lokal 2020-05-29  17650.0

[29332 rows x 3 columns]
<class 'pandas.core.frame